In [93]:
import requests 
import json
import pprint as pp # pretty print

import aries_cloudagent.wallet.crypto as crypto
import base64
import base58
import uuid
import jupyter_util as util

## Enter Invite Url:

In [94]:
agent1 = {}
plugin = {}
endpoint = "http://localhost:8150/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent1['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent1['url'])

http://agent1.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiYzBhZWU1YTItNDk2Yy00NzRjLTk2ZGUtZDI1ZWVjZWY5YTdmIiwgImxhYmVsIjogIk1haW4gKGFkbWluKSIsICJzZXJ2aWNlRW5kcG9pbnQiOiAiaHR0cDovL2FnZW50MS5sb2NhbGhvc3QiLCAicmVjaXBpZW50S2V5cyI6IFsiN1pYR05Ic2NkYmJrWVY4NFc2VWJNZjJWbVJvQ2Nrc3pzU1ZUUjRRVXlIWTUiXX0=


In [95]:
agent2 = {}
endpoint = "http://localhost:8151/connections/create-admin-invitation-url"
adminInvitationResponse = requests.post(endpoint)
adminInvitationResponse = adminInvitationResponse.text
agent2['url'] = json.loads(adminInvitationResponse)['invitation_url']
print(agent2['url'])

http://agent2.localhost?c_i=eyJAdHlwZSI6ICJkaWQ6c292OkJ6Q2JzTlloTXJqSGlxWkRUVUFTSGc7c3BlYy9jb25uZWN0aW9ucy8xLjAvaW52aXRhdGlvbiIsICJAaWQiOiAiNzQxNjNkMDEtZDI1Ni00ZTg0LTlhOTUtYWIyOGM5ODQ3ZGEzIiwgImxhYmVsIjogIkNsaWVudCAoYWRtaW4pIiwgInNlcnZpY2VFbmRwb2ludCI6ICJodHRwOi8vYWdlbnQyLmxvY2FsaG9zdCIsICJyZWNpcGllbnRLZXlzIjogWyIzcVdtZktQNnQzbXdjRzJYakVCUjZOTVEyVjhUamZ2M2Z5NVZWUmdSd3VoRSJdfQ==


In [96]:
agent1 = util.processInviteUrl(agent1['url'])
agent2 = util.processInviteUrl(agent2['url'])
print("Decoded invitation:\n")
pp.pprint(agent2['invite'])

Decoded invitation:

{'@id': '74163d01-d256-4e84-9a95-ab28c9847da3',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
 'label': 'Client (admin)',
 'recipientKeys': ['3qWmfKP6t3mwcG2XjEBR6NMQ2V8Tjfv3fy5VVRgRwuhE'],
 'serviceEndpoint': 'http://agent2.localhost'}


In [97]:
### Generating a did key using aries crypto module
# Create a key pair with random seed 
# Order: 0 - public key, 1 - secret / pritvate key
plugin['keypair'] = crypto.create_keypair()
# it seems to me that did is an encoded sub string of public key
# or maybe just some arbitrary random number
did = plugin['keypair'][0][:16]
# final version of our did
plugin['did'] = base58.b58encode(did).decode("ascii")
# encoding keys to base58
plugin['public_key_b58'] = \
    base58.b58encode(plugin['keypair'][0]).decode("ascii")
plugin['private_key_b58'] = \
    base58.b58encode(plugin['keypair'][1]).decode("ascii")

print("Private key: \n", plugin['private_key_b58'])
print("\nPublic key: \n", plugin['public_key_b58'])

Private key: 
 5tDJTFbwjtRnbXqbYuzH6yB69X3qp4Nx3qY5jxFXYHMHx5aUcDVQN9fveGroYc1AefkAbeWpfhkQWmiv76KLPe71

Public key: 
 EJFAZD6HsgkrvzMt5NBmSeNW6nbYtmxcadRL4YPesdiw


**Sending a connection request to acapy**

In [98]:
   
agent1['connection'] = util.connectWithAcapy(agent1, plugin)
agent2['connection'] = util.connectWithAcapy(agent2, plugin)
# print("Decoded acapy response: \n")
# pp.pprint(agent1['conn'])

**Unpacking connection data embedded in the response**

In [99]:
agent1['connection_details'] = util.decodeConnectionDetails(agent1['connection'])
agent2['connection_details'] = util.decodeConnectionDetails(agent2['connection'])

**Adding the connection to connection list**

In [100]:
connections = [
    {
        "label": agent1['invite']['label'],
        "DIDDoc": agent1['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
    {
        "label": agent2['invite']['label'],
        "DIDDoc": agent2['connection_details']['DIDDoc'],
        "myKey": plugin['keypair'][1]
    },
]

pp.pprint(connections)

[{'DIDDoc': {'@context': 'https://w3id.org/did/v1',
             'authentication': [{'publicKey': 'did:sov:DgqBNsboNv24GsSfvrbFTr#1',
                                 'type': 'Ed25519SignatureAuthentication2018'}],
             'id': 'did:sov:DgqBNsboNv24GsSfvrbFTr',
             'publicKey': [{'controller': 'did:sov:DgqBNsboNv24GsSfvrbFTr',
                            'id': 'did:sov:DgqBNsboNv24GsSfvrbFTr#1',
                            'publicKeyBase58': '7v44NW5sMhZ69zvfx252rhCV8yvp69pLZnfGFDf3D6bq',
                            'type': 'Ed25519VerificationKey2018'}],
             'service': [{'id': 'did:sov:DgqBNsboNv24GsSfvrbFTr;indy',
                          'priority': 0,
                          'recipientKeys': ['7v44NW5sMhZ69zvfx252rhCV8yvp69pLZnfGFDf3D6bq'],
                          'serviceEndpoint': 'http://agent1.localhost',
                          'type': 'IndyAgent'}]},
  'label': 'Main (admin)',
  'myKey': b'\xf4;9\xd7|\xbcp\xe3\xc3\xc4\xde\xe6\xb6\xfd\x1a\xab{\x8

### FEATURE DISCOVERY

In [101]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/query",
    query="*"
)
response = util.sendMessage(message, connections[0])
pp.pprint(response)

{'@id': '24133cc7-dda2-4e68-90bf-bba1912bffbf',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/discover-features/1.0/disclose',
 'protocols': [{'pid': 'https://didcomm.org/introduction-service/0.1'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/action-menu/1.0',
                'roles': ['provider']},
               {'pid': 'https://didcomm.org/trust_ping/1.0'},
               {'pid': 'https://didcomm.org/connections/1.0'},
               {'pid': 'https://didcomm.org/issue-credential/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/introduction-service/0.1'},
               {'pid': 'https://didcomm.org/basicmessage/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/notification/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/trust_ping/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/basicmessage/1.0'},
               {'pid': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0'},
               {'pid

In [108]:
message = util.buildMessage(
    'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/add',
    payload="{aaaaa}: bbdb"
)
response = util.sendMessage(message, connections[0])
pp.pprint(response)


{'@id': 'c8265e38-8aca-4a68-a6fa-e9f7874191b8',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/add',
 'hashid': '5fee3a425c19413ab7ab6d4546794def443d9e9ca109802c424abea7a6411498',
 'payload': '{aaaaa}: bbdb',
 '~thread': {'thid': '87784949-e489-4b37-bbbe-48be6b759ca0'}}


In [110]:
message = util.buildMessage(
    'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/get',
    hashid="5fee3a425c19413ab7ab6d4546794def443d9e9ca109802c424abea7a6411498"
)
response = util.sendMessage(message, connections[0])
pp.pprint(response)


{'@id': '56ac06d5-b7f8-4916-be46-fba16566a6f2',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/acapy-plugin/1.0/get',
 'hashid': '5fee3a425c19413ab7ab6d4546794def443d9e9ca109802c424abea7a6411498',
 'payload': '{aaaaa}: bbdb',
 '~thread': {'thid': '7134b2fb-ed0a-4cbd-970a-def408431162'}}


In [103]:
message = util.buildMessage(
    'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-invitations/0.1/create',
    label="Agent1ToAgent2",
    alias="Agent1ToAgent2",
    auto_accept=True 
)
response = util.sendMessage(message, connections[0])
pp.pprint(response)

{'@id': 'ad2d2052-0e99-497a-b029-538b2526ae07',
 '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/notification/1.0/problem-report',
 'explain-ltxt': 'Unrecognized message type '
                 'https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-invitations/0.1/create',
 '~thread': {'thid': 'f123c883-c0bd-4989-8a5d-a97d1b8bae68'}}


In [104]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-connections/0.1/receive-invitation",
    accept='none',
    invitation=response['invitation_url'],
)
responsee = util.sendMessage(message, connections[1])
pp.pprint(responsee)


KeyError: 'invitation_url'

In [ ]:
message = util.buildMessage(
    "https://github.com/hyperledger/aries-toolbox/tree/master/docs/admin-connections/0.1/get-list",
)
responseee = util.sendMessage(message, connections[1])
pp.pprint(responseee['connections'])

In [ ]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-list-dids"
)
list_dids = util.sendMessage(message, connections[0])
pp.pprint(list_dids)

In [ ]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-dids/0.1/get-public-did"
)
public_did = util.sendMessage(message, connections[0])
pp.pprint(public_did)


In [ ]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-schema-exchange/1.0/send",
    connection_id="d352f12e-cb92-4046-859a-4f05f4e6b40e",
    content="Test"
)
public_did = util.sendMessage(message, connections[0])
pp.pprint(public_did)


In [ ]:
message = util.buildMessage(
    "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/admin-schema-exchange/1.0/get"
)
public_did = util.sendMessage(message, connections[0])
pp.pprint(public_did)
# "6c6642dc-0372-4414-b5e8-98e5f4a58075"